In [1]:
# !pip install webdriver-manager

In [2]:
# !pip install selenium --upgrade

In [21]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import os
import re
import time
from pprint import pprint
import pandas as pd
from bs4 import BeautifulSoup

# Set Chrome options for headless mode
# chrome_options = Options()
# chrome_options.add_argument('--headless=new')
# chrome_options.add_argument('--headless')

options = Options()
options.add_argument("--headless=new")
options.add_argument("user-agent=Chrome/80.0.3987.132")
options.add_argument("--window-size=1920,1080")
# driver = webdriver.Chrome()
# Specify the ChromeDriver version
service = Service(ChromeDriverManager().install())
# Initialize SafariDriver
# driver = webdriver.Safari()

driver = webdriver.Chrome(service=service, options=options)

# Set the URL of the website
url = 'https://www.homes.com/homes-for-sale/?sk=bKQGNiTZM-2TOnjzdIpHc9r8fvsOljeHkDTek1zs6TM&bb=-2vih3-xpOzh2n_tiO'

def listResponse(url):
    driver.get(url)
    time.sleep(0.2)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    return soup

# Open the website
# driver.get(url)
# source = driver.page_source
soup = listResponse(url)
# soup = BeautifulSoup(source, "html.parser")

In [22]:
# pprint(soup)

In [23]:
house_listings = list(soup.select('div.for-sale-content-container'))
house_listings = [BeautifulSoup(str(tag), "lxml") for tag in house_listings]

In [24]:
num_pages = int(soup.select_one('span.pageRange').text.split('of')[1].strip())
base_url = "https://www.homes.com/homes-for-sale/p"
page_links = [f"{base_url}{page_number}/?sk=bKQGNiTZM-2TOnjzdIpHc9r8fvsOljeHkDTek1zs6TM&bb=-2vih3-xpOzh2n_tiO" for page_number in range(1, num_pages + 1)]
cauldron = [listResponse(link) for link in page_links]

In [50]:
house_listings = [list(soup.find_all('div', class_='for-sale-content-container')) for soup in cauldron]

In [56]:
full_listings = [html for soup in house_listings for html in soup]

In [57]:
len(full_listings)

691

In [58]:
# soup.select('div.for-sale-content-container')

In [59]:
df_list = [
    {
        "Agent Full Name": listing.find('span', class_="agent-name").text if listing.find('span', class_="agent-name") else "Data Not Found",
        "Agent First Name": listing.find('span', class_="agent-name").text.split(' ')[0] if listing.find('span', class_="agent-name") else "Data Not Found",
        "Agent Last Name": listing.find('span', class_="agent-name").text.split(' ')[1] if listing.find('span', class_="agent-name") else "Data Not Found",
        "Agency": listing.find('span', class_="agency-name").text if listing.find('span', class_="agency-name") else "Data Not Found",
        "Listing Address": listing.find('p', class_='property-name').text if listing.find('p', class_='property-name') else "Data Not Found",
        "City": listing.find('p', class_='property-name').text.split(', ')[1] if len(listing.find('p', class_='property-name').text.split(', '))>1 else "Data Not Found",
        "Listing Price": listing.find('p', class_='price-container').contents[0].strip() if listing.find('p', class_='price-container') else "Data Not Found",
        "Link": "https://www.homes.com" + listing.find('a').get('href') if listing.find('a').get('href') else "Data Not Found"
    }
    for listing in full_listings
]

In [60]:
df = pd.DataFrame(df_list)
df

,Agent Full Name,Agent First Name,Agent Last Name,Agency,Listing Address,City,Listing Price,Link
0,Lisa Drake,Lisa,Drake,Lisa Drake Properties,"5847 E El Monte Way, Fresno, CA 93727",Fresno,"$289,999",https://www.homes.com/property/5847-e-el-monte...
1,Brooke Clevenger,Brooke,Clevenger,RE/MAX Success,"1774 Firestone Dr, Tulare, CA 93274",Tulare,"$615,000",https://www.homes.com/property/1774-firestone-...
2,Rita Amaro,Rita,Amaro,Rise Real Estate,"5133 W Lamona Ave, Fresno, CA 93722",Fresno,"$625,000",https://www.homes.com/property/5133-w-lamona-a...
3,Paul Gestic,Paul,Gestic,Fresno Income Properties,"21723 Westmere Ln, Friant, CA 93626",Friant,"$1,200,000",https://www.homes.com/property/21723-westmere-...
4,Brandon Gonzales,Brandon,Gonzales,Iron Key Real Estate,"1979 N Elm Ave, Clovis, CA 93619",Clovis,"$950,000",https://www.homes.com/property/1979-n-elm-ave-...
...,...,...,...,...,...,...,...,...
686,Francine Hipskind,Francine,Hipskind,"Wendy Little Properties, Inc.","2039 W Sunnyview Ave, Visalia, CA 93291",Visalia,"$539,000",https://www.homes.com/property/2039-w-sunnyvie...
687,Loraine Greene,Loraine,Greene,Miramar International-Riverwal,"1704 Santa Cruz Dr W, Tulare, CA 93274",Tulare,"$395,000",https://www.homes.com/property/1704-santa-cruz...
688,Shawn Cardoza,Shawn,Cardoza,Legacy Real Estate Inc,"1502 W Sierra Dr, Visalia, CA 93291",Visalia,"$779,900",https://www.homes.com/property/1502-w-sierra-d...
689,Satinder Malhi,Satinder,Malhi,Realty One Group Action,"3065 N Marty Ave Unit 118, Fresno, CA 93722",Fresno,"$313,999",https://www.homes.com/property/3065-n-marty-av...


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691 entries, 0 to 690
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Agent Full Name   691 non-null    object
 1   Agent First Name  691 non-null    object
 2   Agent Last Name   691 non-null    object
 3   Agency            691 non-null    object
 4   Listing Address   691 non-null    object
 5   City              691 non-null    object
 6   Listing Price     691 non-null    object
 7   Link              691 non-null    object
dtypes: object(8)
memory usage: 43.3+ KB
